In [ ]:
!pip install iterative-stratification
!pip install scikit-multilearn
import pandas as pd
from collections import defaultdict 
from tqdm import tqdm
import numpy as np
import csv
from collections import OrderedDict
df = pd.read_excel('phenotypic test results.xlsx')

In [0]:
s = set()
for c in df.columns[1:]:
  s.update(pd.unique(df[c]))
s

{'I', 'IR', 'IS', 'R', 'RS', 'S', 'SI', 'SR', nan}

In [0]:
df = df.replace('I', np.nan)
df = df.replace('IR', 1)
df = df.replace('R', 1)
df = df.replace('RS', 1)
df = df.replace('IS', 0)
df = df.replace('S', 0)
df = df.replace('SI', 0)
df = df.replace('SR', 0)
df0 = pd.DataFrame(columns=df.to_numpy().T[0], index=[df.columns[1:]], data=df.to_numpy().T[1:])

In [0]:
filenames = []
for x in range(0, 24):
  filenames.append('mutations/'+str(x)+'_1.csv')

d_1 = OrderedDict()
for fname in filenames:
  reader = csv.reader(open(fname, 'r'))
  for row in reader:
      d_1[row[0]] = np.array(list(map(int, row[1:])))

In [0]:
l_1 = set()
for k in d_1:
  l_1.update(list(d_1[k]))
dict_number_mutation = dict(zip(range(0, len(l_1)), sorted(l_1)))
dict_mutation_number = dict(zip(sorted(l_1), range(0, len(l_1))))
array_1 = np.zeros((len(d_1), len(l_1)))

for i, k in enumerate(d_1):
  for j in d_1[k]:
    array_1[i, dict_mutation_number[j]] = 1

X = pd.DataFrame(data=array_1, index=d_1.keys(), columns=l_1)

In [0]:
u, c = np.unique(df0.columns, return_counts=True)
dup = u[c > 1]
df_dulicated = df0[dup]
df0 = df0.drop(columns=dup)

In [0]:
for x in dup:
  l1 = df_dulicated[x].to_numpy(dtype=np.float64)[:, 0]
  l2 = df_dulicated[x].to_numpy(dtype=np.float64)[:, 1]
  if sum(np.isnan(l1)) > sum(np.isnan(l2)):
    df0[x] = l2
  else:
    df0[x] = l1
y = df0[d_1.keys()]

In [0]:
y = y.loc[['INH', 'EMB', 'RIF']].dropna(1)
y_array = y.to_numpy().T.astype('int')
X_array = X.loc[y.columns].to_numpy().astype('int')

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.metrics import roc_auc_score, make_scorer

roc_auc_macro = make_scorer(roc_auc_score, average='macro')
mskf = MultilabelStratifiedKFold(n_splits=5)
parameters_rf = {'n_estimators':[10,20,30,100], 'max_features':['auto','log2'], 'min_samples_split':[2,4,8], 'bootstrap':[True, False]}
clf_rnn = GridSearchCV(RandomForestClassifier(), parameters_rf, scoring=roc_auc_macro, cv=mskf)
clf_rnn.fit(X_array, y_array)

GridSearchCV(cv=MultilabelStratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_sample...
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [0]:
from skmultilearn.adapt import MLkNN

def score(estimator, X, y):
    y_pred = estimator.predict(X).toarray()
    return roc_auc_score(y, y_pred, average='macro')

parameters_knn = {'k': [1, 3, 5], 's': [0.5, 0.7, 1.0]}
knn = MLkNN()
clf_knn = GridSearchCV(knn, parameters_knn, scoring=score, cv=mskf)
clf_knn.fit(X_array, y_array)

GridSearchCV(cv=MultilabelStratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             error_score=nan,
             estimator=MLkNN(ignore_first_neighbours=0, k=10, s=1.0),
             iid='deprecated', n_jobs=None,
             param_grid={'k': [1, 3, 5], 's': [0.5, 0.7, 1.0]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=<function score at 0x7f22ff632e18>, verbose=0)

In [0]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

knn_results = {"accuracy":0, "roc_auc":0, "f1":0, "recall":0, "precision":0}
rf_results = {"accuracy":0, "roc_auc":0, "f1":0, "recall":0, "precision":0}

for train_index, test_index in tqdm(mskf.split(X_array, y_array)):
    X_train, X_test = X_array[train_index], X_array[test_index]
    y_train, y_test = y_array[train_index], y_array[test_index]
    clf_knn.best_estimator_.fit(X_train, y_train)
    y_pred_knn = clf_knn.best_estimator_.predict(X_test).toarray()
    clf_rnn.best_estimator_.fit(X_train, y_train)
    y_pred_rf = clf_rnn.best_estimator_.predict(X_test)
    for y_pred, d in zip([y_pred_knn, y_pred_rf], [knn_results, rf_results]):
        d["accuracy"] += accuracy_score(y_pred, y_test) / 5
        d["precision"] += precision_score(y_pred, y_test, average='macro') / 5
        d["recall"] += recall_score(y_pred, y_test, average='macro') / 5
        d["roc_auc"] += roc_auc_score(y_pred, y_test, average='macro') / 5
        d["f1"] += f1_score(y_pred, y_test, average='macro') / 5



0it [00:00, ?it/s]

1it [00:25, 25.86s/it]

2it [00:52, 25.97s/it]

3it [01:18, 26.02s/it]

4it [01:43, 25.90s/it]

5it [02:09, 25.86s/it]


In [0]:
print(knn_results, rf_results)

{'accuracy': 0.6480045656516245, 'roc_auc': 0.7690401673763242, 'f1': 0.8712799847109658, 'recall': 0.8439980242469207, 'precision': 0.9008250390575517} {'accuracy': 0.6823244058538176, 'roc_auc': 0.8081731485707767, 'f1': 0.8908786211546577, 'recall': 0.865535252770939, 'precision': 0.9190801534575529}


In [0]:
from sklearn.multioutput import ClassifierChain
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')
    
chains_results = pd.DataFrame({'accuracy': [0.0, 0.0, 0.0, 0.0, 0.0], 'recall': [0.0, 0.0, 0.0, 0.0, 0.0], 
                               'precision': [0.0, 0.0, 0.0, 0.0, 0.0], 'roc_auc': [0.0, 0.0, 0.0, 0.0, 0.0], 
                               'f1': [0.0, 0.0, 0.0, 0.0, 0.0]}, index=[10, 20, 30, 40, 50])

for train_index, test_index in tqdm(mskf.split(X_array, y_array)):
    X_train, X_test = X_array[train_index], X_array[test_index]
    y_train, y_test = y_array[train_index], y_array[test_index]
    for t in tqdm([10, 20, 30, 40, 50]):
        base_lr = LogisticRegression()
        chains = [ClassifierChain(base_lr, order='random', random_state=i) for i in range(t)]
        for chain in chains:
            chain.fit(X_train, y_train)
        y_pred = np.array([chain.predict(X_test) for chain in chains]).mean(axis=0)
        y_pred[y_pred >= 0.5] = 1
        y_pred[y_pred < 0.5] = 0
        print(accuracy_score(y_pred, y_test))
        chains_results.at[t, 'accuracy'] = chains_results.at[t, 'accuracy'] + accuracy_score(y_pred, y_test) / 5
        chains_results.at[t, "precision"] = chains_results.at[t, "precision"] + precision_score(y_pred, y_test, average='macro') / 5
        chains_results.at[t, "recall"] = chains_results.at[t, "recall"] + recall_score(y_pred, y_test, average='macro') / 5
        chains_results.at[t, "roc_auc"] = chains_results.at[t, "roc_auc"] + roc_auc_score(y_pred, y_test, average='macro') / 5
        chains_results.at[t, "f1"] = chains_results.at[t, "f1"] + f1_score(y_pred, y_test, average='macro') / 5
        print(chains_results)











0it [00:00, ?it/s]










  0%|          | 0/5 [00:00<?, ?it/s]










 20%|██        | 1/5 [01:45<07:00, 105.24s/it]

0.7285067873303167
    accuracy    recall  precision   roc_auc        f1
10  0.145701  0.176643   0.186794  0.168524  0.181525
20  0.000000  0.000000   0.000000  0.000000  0.000000
30  0.000000  0.000000   0.000000  0.000000  0.000000
40  0.000000  0.000000   0.000000  0.000000  0.000000
50  0.000000  0.000000   0.000000  0.000000  0.000000













 40%|████      | 2/5 [05:19<06:54, 138.05s/it]

0.7285067873303167
    accuracy    recall  precision   roc_auc        f1
10  0.145701  0.176643   0.186794  0.168524  0.181525
20  0.145701  0.176253   0.186794  0.168331  0.181300
30  0.000000  0.000000   0.000000  0.000000  0.000000
40  0.000000  0.000000   0.000000  0.000000  0.000000
50  0.000000  0.000000   0.000000  0.000000  0.000000













 60%|██████    | 3/5 [10:23<06:15, 187.86s/it]

...

0.7117117117117117
    accuracy    recall  precision   roc_auc        f1
10  0.710301  0.889692   0.916267  0.834625  0.902506
20  0.709400  0.889578   0.916140  0.834946  0.902372
30  0.709404  0.889833   0.915639  0.834910  0.902272
40  0.567062  0.709568   0.734535  0.668599  0.721597
50  0.567062  0.709568   0.734535  0.668599  0.721597













 80%|████████  | 4/5 [16:25<04:03, 243.89s/it]

0.7117117117117117
    accuracy    recall  precision   roc_auc        f1
10  0.710301  0.889692   0.916267  0.834625  0.902506
20  0.709400  0.889578   0.916140  0.834946  0.902372
30  0.709404  0.889833   0.915639  0.834910  0.902272
40  0.709404  0.889833   0.915639  0.834910  0.902272
50  0.567062  0.709568   0.734535  0.668599  0.721597













100%|██████████| 5/5 [24:50<00:00, 298.00s/it]










5it [2:04:57, 1499.55s/it]

0.7117117117117117
    accuracy    recall  precision   roc_auc        f1
10  0.710301  0.889692   0.916267  0.834625  0.902506
20  0.709400  0.889578   0.916140  0.834946  0.902372
30  0.709404  0.889833   0.915639  0.834910  0.902272
40  0.709404  0.889833   0.915639  0.834910  0.902272
50  0.709404  0.889833   0.915639  0.834910  0.902272


In [0]:
chains_results

,accuracy,recall,precision,roc_auc,f1
10,0.710301,0.889692,0.916267,0.834625,0.902506
20,0.709400,0.889578,0.916140,0.834946,0.902372
30,0.709404,0.889833,0.915639,0.834910,0.902272
40,0.709404,0.889833,0.915639,0.834910,0.902272
50,0.709404,0.889833,0.915639,0.834910,0.902272
